In [1]:
import os
import warnings
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd

In [44]:
# 구글 드라이브 연동
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [7]:
faqs = pd.read_csv(os.path.join('data','kor_elec_faq2.csv'), encoding='CP949')
faqs

,순번,조회수,업무분류,업무분류명,등록일,수정일,제목,내용,FAQ분류
0,554,10912,20023,전기사용:할인요금,2003-02-14,2018-08-07,1주택수가구 요금을 적용받고 있는데 가구수가 감소하면 어떻게 해야 합니까.,1주택수가구 요금을 적용받던 고객이 세입자 전출등 가구수가 감소하는 경우에는 1개월...,전기요금:전기요금 계산
1,229,3875,20100,전기사용:신규업무:공급방법/장소,2003-02-14,2019-02-11,고객이 임시공급설비 설치공사비를 부담하여 한전에서 임시공급설비를 시설하였을 경우의 ...,임시공급설비를 한전에서 설치할 경우에는 상시전력 공급설비의 수급지점과 동일하게하여 ...,전기사용:전기사용신청
2,535,20176,20100,전기사용:신규업무:공급방법/장소,2003-02-14,2018-08-07,"변압기공동이용(모자거래)이란 무엇이며, 요금계산은 어떻게 합니까.",동일 전기사용장소내에 전기사용용도(계약종별)가 다르거나 회계단위가 상이한 2이상의 ...,전기사용:전기사용계약
3,117,4461,20100,전기사용:신규업무:공급방법/장소,2003-02-14,2018-08-07,공장을 새로 짓는데 옆 공장에서 변압기용량이 남는다고 하여 일부를 연결하여 같이 사...,동일한 전기사용 장소내에서 전기를 사용하고 계약종별 또는 회계단위가 상이한 2이상의...,전기사용:전기사용계약
4,143,27540,20100,전기사용:신규업무:공급방법/장소,2003-02-14,2018-08-07,전기설비중 냉·난방기기가 있는데 계약전력은 어떻게 산정합니까.,여름철에만 사용하는 설비와 겨울철에만 사용하는 설비가 1Set로 구성되어 냉난방용 ...,전기사용:전기사용신청
...,...,...,...,...,...,...,...,...,...
346,1246,1989,20224,시스템:대외시스템:사이버지점,2007-12-05,2017-01-03,전기공급약관을 보고 싶습니다.,전기공급약관은 `전력거래약관(새창열림)`href=`http://cyber.kepco...,사이버지점:사이버지점 이용
347,1468,4054,20224,시스템:대외시스템:사이버지점,2013-06-28,2017-01-03,법인회원으로 가입하는 방법을 알려주세요.,"개인회원으로 가입하신 후, `기업서비스이용신청(새창열림)`href=`https://...",사이버지점:사이버지점 이용
348,1469,2880,20224,시스템:대외시스템:사이버지점,2013-06-28,2017-01-03,전기요금을 미리 계산해 볼 수 있나요.,`전기요금계산기(새창열림)`href=`http://cyber.kepco.co.kr/...,사이버지점:사이버지점 이용
349,1470,6323,20224,시스템:대외시스템:사이버지점,2013-06-28,2017-01-03,아파트 관리사무소입니다. 종합계약과 단일계약을 비교해보고 싶습니다.,`아파트·오피스텔요금비교(새창열림)`href=`http://cyber.kepco.c...,사이버지점:사이버지점 이용


In [8]:
faqs[['순번', '제목', '내용']]

,순번,제목,내용
0,554,1주택수가구 요금을 적용받고 있는데 가구수가 감소하면 어떻게 해야 합니까.,1주택수가구 요금을 적용받던 고객이 세입자 전출등 가구수가 감소하는 경우에는 1개월...
1,229,고객이 임시공급설비 설치공사비를 부담하여 한전에서 임시공급설비를 시설하였을 경우의 ...,임시공급설비를 한전에서 설치할 경우에는 상시전력 공급설비의 수급지점과 동일하게하여 ...
2,535,"변압기공동이용(모자거래)이란 무엇이며, 요금계산은 어떻게 합니까.",동일 전기사용장소내에 전기사용용도(계약종별)가 다르거나 회계단위가 상이한 2이상의 ...
3,117,공장을 새로 짓는데 옆 공장에서 변압기용량이 남는다고 하여 일부를 연결하여 같이 사...,동일한 전기사용 장소내에서 전기를 사용하고 계약종별 또는 회계단위가 상이한 2이상의...
4,143,전기설비중 냉·난방기기가 있는데 계약전력은 어떻게 산정합니까.,여름철에만 사용하는 설비와 겨울철에만 사용하는 설비가 1Set로 구성되어 냉난방용 ...
...,...,...,...
346,1246,전기공급약관을 보고 싶습니다.,전기공급약관은 `전력거래약관(새창열림)`href=`http://cyber.kepco...
347,1468,법인회원으로 가입하는 방법을 알려주세요.,"개인회원으로 가입하신 후, `기업서비스이용신청(새창열림)`href=`https://..."
348,1469,전기요금을 미리 계산해 볼 수 있나요.,`전기요금계산기(새창열림)`href=`http://cyber.kepco.co.kr/...
349,1470,아파트 관리사무소입니다. 종합계약과 단일계약을 비교해보고 싶습니다.,`아파트·오피스텔요금비교(새창열림)`href=`http://cyber.kepco.c...


In [9]:
!pip install konlpy

In [10]:
!pip install --upgrade h5py

  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0


In [11]:
#형태소 분석
import jpype
from konlpy.tag import Kkma

In [12]:
# kkma = Kkma()
# filter_kkma = ['NNG',  #보통명사
#              'NNP',  #고유명사
#              'OL' ,  #외국어
#             ]

# def tokenize_kkma(doc):
#     jpype.attachThreadToJVM()
#     token_doc = ['/'.join(word) for word in kkma.pos(doc)]
#     return token_doc

# def tokenize_kkma_noun(doc):
#     jpype.attachThreadToJVM()
#     token_doc = ['/'.join(word) for word in kkma.pos(doc) if word[1] in filter_kkma]
#     return token_doc


In [27]:
#튜닝
kkma = Kkma()
filter_kkma = ['NNG',  #보통명사
             'NNP',  #고유명사
             'OL' ,  #외국어
             'VA','VV','VXV'
            ]

def tokenize_kkma(doc):
    jpype.attachThreadToJVM()
    token_doc = ['/'.join(word) for word in kkma.pos(doc)]
    return token_doc

def tokenize_kkma_noun(doc):
    jpype.attachThreadToJVM()
    token_doc = ['/'.join(word) for word in kkma.pos(doc) if word[1] in filter_kkma]
    return token_doc


In [ ]:
#자바를 안하면 나오는 에러
#from konlpy.tag import Kkma
#kkma = Kkma()
#def tokenize_kkma(doc):
#    token_doc = ['/'.join(word) for word in kkma.pos(doc) ]
#    return token_doc

In [28]:
tokenize_kkma_noun(faqs['제목'][0])

['주택/NNG',
 '수가/NNG',
 '구/NNG',
 '요금/NNG',
 '적용/NNG',
 '받/VV',
 '있/VXV',
 '가구/NNG',
 '수가/NNG',
 '감소/NNG',
 '어떻/VA',
 '하/VV',
 '하/VV']

In [14]:
len(faqs)

351

In [25]:
# 리스트에서 각 문장부분 토큰화
token_faqs = []
for i in range(len(faqs)):
    token_faqs.append([tokenize_kkma_noun(faqs['제목'][i]), i ])
    # token_faqs.append([tokenize_kkma_noun(faqs['제목'][i]), faqs['순번'][i]])

# Doc2Vec에서 사용하는 태그문서형으로 변경
tagged_faqs = [TaggedDocument(d, [int(c)]) for d, c in token_faqs]
# tagged_faqs = [TaggedDocument(d, [c]) for d, c in token_faqs]

In [26]:
tagged_faqs[:5]

[TaggedDocument(words=['주택/NNG', '수가/NNG', '구/NNG', '요금/NNG', '적용/NNG', '가구/NNG', '수가/NNG', '감소/NNG'], tags=[0]),
 TaggedDocument(words=['고객/NNG', '임시/NNG', '공급/NNG', '설비/NNG', '설치/NNG', '공사비/NNG', '부담/NNG', '한전/NNG', '임시/NNG', '공급/NNG', '설비/NNG', '시설/NNG', '경우/NNG', '임시/NNG', '공급/NNG', '설비/NNG', '유지/NNG', '보수/NNG', '책임/NNG', '한계점/NNG'], tags=[1]),
 TaggedDocument(words=['변압기/NNG', '공동/NNG', '이용/NNG', '모자/NNG', '거래/NNG', '요금/NNG', '계산/NNG'], tags=[2]),
 TaggedDocument(words=['공장/NNG', '옆/NNG', '공장/NNG', '변압기/NNG', '용량/NNG', '일부/NNG', '연결/NNG', '사용/NNG'], tags=[3]),
 TaggedDocument(words=['전기/NNG', '설비/NNG', '중/NNG', '냉/NNG', '난방/NNG', '기기/NNG', '계약/NNG', '전력/NNG', '산정/NNG'], tags=[4])]

In [33]:
    # make model
    import multiprocessing
    import tensorflow as tf
    with tf.device('/device:GPU:0'):
        cores = multiprocessing.cpu_count()
        d2v_faqs = doc2vec.Doc2Vec(vector_size=20, #100
                                    alpha=0.025,
                                    min_alpha=0.025,
                                    hs=1,
                                    negative=0,
                                    dm=0,
                                    window=3,
                                    dbow_words = 1,
                                    min_count = 1,
                                    workers = cores,
                                    seed=0,
                                    epochs=100)
        d2v_faqs.build_vocab(tagged_faqs)

        # train document vectors
        for epoch in range(50):
            d2v_faqs.train(tagged_faqs,
                                    total_examples = d2v_faqs.corpus_count,
                                    epochs = d2v_faqs.epochs
                                    )
            d2v_faqs.alpha -= 0.0025 # decrease the learning rate
            d2v_faqs.min_alpha = d2v_faqs.alpha # fix the learning rate, no decay

In [19]:
d2v_faqs.vocabulary.sample

AttributeError: 'Doc2Vec' object has no attribute 'vocabulary'

In [36]:
# test_string = "건물을 새로 지을 때 임시전력은 어떻게 신청하나요"
test_string = "변압기공동이용(모자거래)이란 무엇이며, 요금계산은 어떻게 합니까"
tokened_test_string = tokenize_kkma_noun(test_string)
tokened_test_string

['변압기/NNG',
 '공동/NNG',
 '이용/NNG',
 '모자/NNG',
 '거래/NNG',
 '요금/NNG',
 '계산/NNG',
 '어떻/VA',
 '하/VV']

In [37]:
topn = 5
test_vector = d2v_faqs.infer_vector(tokened_test_string)
result = d2v_faqs.docvecs.most_similar([test_vector], topn=topn)
print(result)

for i in range(topn):
    # print("{}위. {}, {}".format(i+1, result[i][1], faqs[faqs['순번']==int(result[i][0])]['제목']))
    print("{}위. {}, {}{}".format(i+1, result[i][1], result[i][0], faqs['제목'][result[i][0]]))

[(187, 0.721482515335083), (2, 0.6414189338684082), (330, 0.544560968875885), (188, 0.5432237982749939), (306, 0.49761253595352173)]
1위. 0.721482515335083, 187역률이란 무엇이며 어떻게 계산합니까.
2위. 0.6414189338684082, 2변압기공동이용(모자거래)이란 무엇이며, 요금계산은 어떻게 합니까.
3위. 0.544560968875885, 330절연저항이란 무엇이며, 옥내절연저항치는 얼마 이상 유지해야 합니까.
4위. 0.5432237982749939, 188공장설비 증가로 저압에서 특별고압으로 공급방식을 변경하였습니다. 이달의 역률 적용은 저압과 특별고압으로 구분하여 적용합니까.
5위. 0.49761253595352173, 306우리집 앞에 설치되어 있는 전주의 변압기에서 소음이 크게 나는데 이상이 있는 것 아닙니까.


<ipython-input-37-475feba358c2>:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  result = d2v_faqs.docvecs.most_similar([test_vector], topn=topn)


In [39]:
# 성능 측정
# raten = 5 #정확도 = 0.5128205128205128 % (180/351 )
raten = 1 #정확도 = 0.24216524216524216 % (85/351 ) 
found = 0
for i in range(len(faqs)):
    tstr = faqs['제목'][i]
    ttok = tokenize_kkma_noun(tstr)
    tvec = d2v_faqs.infer_vector(ttok)
    re = d2v_faqs.docvecs.most_similar([tvec], topn = raten)
    for j in range(raten):
        if i == re[j][0]: found = found + 1

print("정확도 = {} % ({}/{} )  ".format(found/len(faqs),found, len(faqs)))

<ipython-input-39-9356d78b225d>:9: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  re = d2v_faqs.docvecs.most_similar([tvec], topn = raten)


정확도 = 0.24216524216524216 % (85/351 )  


In [45]:
# 모델 저장

# 모델 1
d2v_faqs.save(os.path.join('data','/d2v_faqs_size100_min1_batch50_epoch100_nounonly_dm0.model'))

#모델 2
# d2v_faqs.save(os.path.join('data','/d2v_faqs_size20_min1_batch50_epoch50_nounonly_dm0.model'))

In [46]:
# 모델 load
d2v_faqs_1 = doc2vec.Doc2Vec.load(os.path.join('data','/d2v_faqs_size100_min1_batch50_epoch100_nounonly_dm0.model'))
d2v_faqs_2 = doc2vec.Doc2Vec.load(os.path.join('data','/d2v_faqs_size20_min1_batch50_epoch50_nounonly_dm0.model'))


In [47]:
test_string = "건물을 새로 지을 때 임시전력은 어떻게 신청하나요"
tokened_test_string = tokenize_kkma_noun(test_string)
tokened_test_string

topn = 5
# 모델 1 추측
test_vector1 = d2v_faqs_1.infer_vector(tokened_test_string)
result1 = d2v_faqs_1.docvecs.most_similar([test_vector1], topn=topn)

# 모델 2 추측
test_vector2 = d2v_faqs_2.infer_vector(tokened_test_string)
result2 = d2v_faqs_2.docvecs.most_similar([test_vector2], topn=topn)

for i in range(topn):
    print("모델 1 {}위. {}, {} {}".format(i+1, result1[i][1], result1[i][0],faqs['제목'][result1[i][0]] ))

print('')
for i in range(topn):
    print("모델 2 {}위. {}, {} {}".format(i+1, result2[i][1], result2[i][0],faqs['제목'][result2[i][0]] ))

모델 1 1위. 0.5144060254096985, 31 아파트 전기사용계약 방법에 대하여 알려주세요.
모델 1 2위. 0.4977785348892212, 266 전기요금 자동이체 신청 및 해지방법을 알려주세요.
모델 1 3위. 0.489153116941452, 243 전기요금은 어떤 방법으로 계산합니까.
모델 1 4위. 0.4829809367656708, 94 장사를 하는데 주택용전력으로 되어 있어 요금부담이 많습니다. 일반용으로 적용받을 수 없습니까.
모델 1 5위. 0.427913635969162, 250 착오로 전기요금을 두 번 냈는데 어떻게 해야 환불 받을 수 있습니까 .

모델 2 1위. 0.5144060254096985, 31 아파트 전기사용계약 방법에 대하여 알려주세요.
모델 2 2위. 0.4977785348892212, 266 전기요금 자동이체 신청 및 해지방법을 알려주세요.
모델 2 3위. 0.489153116941452, 243 전기요금은 어떤 방법으로 계산합니까.
모델 2 4위. 0.4829809367656708, 94 장사를 하는데 주택용전력으로 되어 있어 요금부담이 많습니다. 일반용으로 적용받을 수 없습니까.
모델 2 5위. 0.427913635969162, 250 착오로 전기요금을 두 번 냈는데 어떻게 해야 환불 받을 수 있습니까 .


<ipython-input-47-76d930ccd396>:8: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  result1 = d2v_faqs_1.docvecs.most_similar([test_vector1], topn=topn)
<ipython-input-47-76d930ccd396>:12: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  result2 = d2v_faqs_2.docvecs.most_similar([test_vector2], topn=topn)


In [48]:
# 성능 측정
raten = 1
found1 = 0
found2 = 0

for i in range(len(faqs)):
    tstr = faqs['제목'][i]
    ttok = tokenize_kkma_noun(tstr)
    tvec1 = d2v_faqs_1.infer_vector(ttok)
    re1 = d2v_faqs_1.docvecs.most_similar([tvec1], topn = raten)
    tvec2 = d2v_faqs_2.infer_vector(ttok)
    re2 = d2v_faqs_2.docvecs.most_similar([tvec2], topn = raten)
    for j in range(raten):
        if i == re1[j][0]: found1 = found1 + 1
        if i == re2[j][0]: found2 = found2 + 1

print("모델 1 정확도 = {} % ({}/{} )  ".format(found1/len(faqs),found1, len(faqs)))
print("모델 2 정확도 = {} % ({}/{} )  ".format(found2/len(faqs),found2, len(faqs)))

<ipython-input-48-c65f09a4d934>:10: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  re1 = d2v_faqs_1.docvecs.most_similar([tvec1], topn = raten)
<ipython-input-48-c65f09a4d934>:12: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  re2 = d2v_faqs_2.docvecs.most_similar([tvec2], topn = raten)


모델 1 정확도 = 0.27635327635327633 % (97/351 )  
모델 2 정확도 = 0.27635327635327633 % (97/351 )  
